# Well System Model

![River Base Concept Model](river_and_wells.svg)

# Example description
### Spatial configuration
There is two simulated aquifer which are separated by an aquitard. The model grid consists of 15 rows, 10 columns, and 3 layers.  Uniform grid spacing with a row and column width of 100.0 meters. The top layer is positioned at an elevation of 15.0 meters, while the bottom layers are situated at elevations of -5.0, -10.0, and -15.0 meters, respectively.
### Temporal discretization
The model is designed for a simulation period of 10 days, divided into 120 time steps per stress period, with a time step multiplier of 1.0. The simulation is repeated for a total of 3 periods. The model operates in days as time units and meters as length units.
### Layer Hydraulic properties 
Initial hydraulic conductivity values for the three layers are set at 0.5, 0.000006, and 0.5, respectively.
The vertical anisotropy ratios for the layers are specified as 0.1, 0.2, and 0.3, indicating differences in hydraulic conductivity in the vertical direction.
### Storage properties 
The specific yield is set at 0.2, representing the volumetric ratio of water that drains under the influence of gravity.The specific storage is specified as 0.000001, representing the compressibility of the aquifer.
### Boundary Conditions 
The model starts with an initial hydraulic head of 10.0 meters across the entire domain. 
Two constant head boundaries are established at specific locations: one at the intersection of the first layer, first row, and first column and another at the intersection of the last layer, last row, and last column, each set at a constant head of 10.0 meters.
### Solute transport conditions 
The initial concentration of the substance being transported within the groundwater system set to 1.
The model simulates a point source contamination: at cell (Layer 0, Row 5, Column 1), the initial concentration is set to 10.
Similarly, at cell (Layer 0, Row 6, Column 1), the initial concentration is also set to 10.

# Start setting up the model 

### Magic commands - auto reload of the model each time 

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

### Import from pymf6tools the functions to run, get and visualize simulation results

In [4]:
from pathlib import Path
import numpy as np 
from pymf6.mf6 import MF6
import pandas as pd 
from functools import partial 

from pymf6_tools.base_model import make_model_data
from pymf6_tools.make_model import make_input, run_simulation, get_simulation
from pymf6_tools.plotting import show_heads, show_well_head, show_bcs

In [5]:
from pymf6_tools.plotting import show_heads, show_well_head, show_concentration, show_bcs, show_bot_elevations, show_river_stages, contour_bot_elevations, plot_spec_discharge

## Set model path and name 

In [6]:
model_path = r'models/gwf_transbase'
model_name = 'transbase'

## Run simulation

In [7]:
run_simulation(model_path)

MFDataException: An error occurred in package "None". The error occurred while loading package file in the "load" method.
Additional Information:
(1) File C:\Users\lucialabarca\pymf6-validation\src\notebooks\models\gwf_transbase\mfsim.nam of type nam could not be opened.

### Inspect visualization tools

In [ ]:
show_bcs?

### Inspect the parameters by importing the model results 

In [ ]:
mf6 = MF6(model_path)

In [ ]:
mf6.models.keys()

In [ ]:
gwf_models = mf6.models['gwf6']

In [ ]:
gwf_models.keys()

In [ ]:
gwt_models = mf6.models['gwt6']

In [ ]:
gwt_models.keys()

In [ ]:
gwt = gwt_models['gwt_transbase']

In [ ]:
gwf.packages

In [ ]:
gwt.packages

## Visualization of Input and Output - e.g. Boundary conditions and Heads 

### Boundary Conditions 
Note that you should change the "bc_names" according to the boundary conditions present in the simulation.  
<span style="color:blue">'chd'</span> Constant-head boundary  
<span style="color:blue">'riv-1'</span> River boundary

In [ ]:
show_bcs?

In [ ]:
show_bcs(model_path, model_name, bc_names=('chd','wel'))

### Specific Discharge - Interactive graph (layer and time)
Specify the layer and time of the simulation to be visualized.

In [ ]:
plot_spec_discharge(model_path, model_name, layer=2, times = 300)

In [ ]:
plot_spec_discharge(model_path, model_name, layer=1, times = 300)

### Bottom elevations with Countour lines - Interactive layer 
Specify the layer to be visualized.

### Groundwater level 

In [ ]:
show_heads(model_path, model_name, show_wells=False)

### Concentration

In [ ]:
show_concentration(model_path, model_name, show_wells=True, show_arrows=True)

### Well Head 

In [ ]:
specific_model_data = {
    'model_path': model_path,
    'name': model_name,
}

model_data = make_model_data(specific_model_data)

pymf6_model_data = make_model_data(specific_model_data, base_model_data=model_data)


In [ ]:
wel_coords = model_data['wells']['wel_out']['coords']

In [ ]:
from functools import partial

show_my_well_head = partial(show_well_head,
                            wel_coords=wel_coords,
                            lower_head_limit=0.49,
                            upper_head_limit=0.51)

In [ ]:
show_my_well_head(model_data=pymf6_model_data)